In [2]:
#!pip install awswrangler

In [3]:
#!pip install 'awswrangler[redshift]'

In [77]:
import awswrangler as wr
import boto3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from io import BytesIO # Para el WA

In [5]:
# Config cliente de S3
s3 = boto3.client('s3')

In [6]:
# Setting processing_date
processing_date = datetime.today().strftime('%Y-%m-%d')
processing_date

'2024-04-15'

In [55]:
previous_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
previous_date

'2024-04-14'

In [28]:
database_name = "analytics"
table_name = "daily_check_gp"
df_daily = wr.athena.read_sql_table(
    table=table_name,
    database=database_name,
)

In [66]:
df_daily['date'] = pd.to_datetime(df_daily['date'])

In [67]:
df_daily.head()

,payer,country,date,tx,amount,coupon_count,gp,id_main_branch,id_country,day
0,TELEDOLAR,COSTA RICA,2021-03-17,122,52049.4500,104,236.2500000,T072,CCR,2021-03-17
1,NAFA,GUINEA,2022-12-14,3,307.0000,0,16.2300000,T303,GUI,2022-12-14
2,BANCO RENDIMENTO,BRAZIL,2022-02-23,907,817032.7100,0,10430.3637000,T056,BRA,2022-02-23
3,BANCOLOMBIA,COLOMBIA,2021-03-17,368,124517.0900,238,1483.8408000,T060,COL,2021-03-17
4,BANCO DAVIVIENDA SALVADORENO (BTS),EL SALVADOR,2021-03-17,41,19854.0000,32,75.6000000,T124,ELS,2021-03-17


In [68]:
df_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115332 entries, 0 to 115331
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   payer           115332 non-null  string        
 1   country         115332 non-null  string        
 2   date            115332 non-null  datetime64[ns]
 3   tx              115332 non-null  Int64         
 4   amount          115332 non-null  object        
 5   coupon_count    115332 non-null  Int64         
 6   gp              115332 non-null  object        
 7   id_main_branch  115332 non-null  string        
 8   id_country      115332 non-null  string        
 9   day             115332 non-null  string        
dtypes: Int64(2), datetime64[ns](1), object(2), string(5)
memory usage: 9.0+ MB


In [14]:
# Bucket
s3_bucket = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
inference_prefix = 'inference_prediction'

In [43]:
# Aca deberia levantar el df con las predicciones
#predict_path = f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/inference_prediction/day={previous_date}/predictions_2d/'
#df_predict = wr.s3.read_parquet(predict_path)

In [57]:
# WA: Leo un archivo dummy que tiene la estructura final
df_predict = pd.read_excel('df_pred.xlsx')

In [59]:
df_predict.drop('Unnamed: 0', axis=1, inplace=True)

In [61]:
df_predict.head()

,processing_date,pred_date,pred,payer_country,model,id_main_branch,id_country,payer,country
0,2024-03-20,2023-12-25,36947.319758,24XORO_MEXICO,MODEL_2d_24XORO_MEXICO.pkl,T314,MEX,24XORO,MEXICO
1,2024-03-20,2023-12-26,43751.589258,24XORO_MEXICO,MODEL_2d_24XORO_MEXICO.pkl,T314,MEX,24XORO,MEXICO
2,2024-03-20,2023-12-25,9590.269894,ABANK (TN)_EL SALVADOR,MODEL_2d_ABANK (TN)_EL SALVADOR.pkl,T282,ELS,ABANK (TN),EL SALVADOR
3,2024-03-20,2023-12-26,8998.724809,ABANK (TN)_EL SALVADOR,MODEL_2d_ABANK (TN)_EL SALVADOR.pkl,T282,ELS,ABANK (TN),EL SALVADOR
4,2024-03-20,2023-12-25,5632.096572,AFEX_CHILE,MODEL_2d_AFEX_CHILE.pkl,T089,CHI,AFEX,CHILE


In [64]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   processing_date  260 non-null    datetime64[ns]
 1   pred_date        260 non-null    datetime64[ns]
 2   pred             260 non-null    float64       
 3   payer_country    260 non-null    object        
 4   model            260 non-null    object        
 5   id_main_branch   260 non-null    object        
 6   id_country       260 non-null    object        
 7   payer            260 non-null    object        
 8   country          260 non-null    object        
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 18.4+ KB


Con los resultados de las predicciones + los valores de amount correspondientes voy a calcular el MAPE 

In [75]:
# Unir los dfs por la columna de fecha
merged_df = pd.merge(df_predict, df_daily, left_on=['pred_date','id_main_branch','id_country', 'payer', 'country'], 
                     right_on=['date','id_main_branch','id_country', 'payer', 'country'])

In [82]:
merged_df.head()

,processing_date,pred_date,pred,payer_country,model,id_main_branch,id_country,payer,country,date,tx,amount,coupon_count,gp,day,abs_error,MAPE
0,2024-03-20,2023-12-25,36947.319758,24XORO_MEXICO,MODEL_2d_24XORO_MEXICO.pkl,T314,MEX,24XORO,MEXICO,2023-12-25,6,2283.00,1,20.1258000,2023-12-25,34664.319758,15.183671
1,2024-03-20,2023-12-26,43751.589258,24XORO_MEXICO,MODEL_2d_24XORO_MEXICO.pkl,T314,MEX,24XORO,MEXICO,2023-12-26,65,42320.71,3,377.6070000,2023-12-26,1430.879258,0.033810
2,2024-03-20,2023-12-25,9590.269894,ABANK (TN)_EL SALVADOR,MODEL_2d_ABANK (TN)_EL SALVADOR.pkl,T282,ELS,ABANK (TN),EL SALVADOR,2023-12-25,4,600.00,0,5.3000000,2023-12-25,8990.269894,14.983783
3,2024-03-20,2023-12-26,8998.724809,ABANK (TN)_EL SALVADOR,MODEL_2d_ABANK (TN)_EL SALVADOR.pkl,T282,ELS,ABANK (TN),EL SALVADOR,2023-12-26,65,16501.00,1,104.4400000,2023-12-26,7502.275191,0.454656
4,2024-03-20,2023-12-25,5632.096572,AFEX_CHILE,MODEL_2d_AFEX_CHILE.pkl,T089,CHI,AFEX,CHILE,2023-12-25,1,280.00,0,3.5000000,2023-12-25,5352.096572,19.114631


In [80]:
# Calcular el error absoluto
merged_df['pred'] = merged_df['pred'].astype(float)
merged_df['amount'] = merged_df['amount'].astype(float)
merged_df['abs_error'] = np.abs(merged_df['pred'] - merged_df['amount'])

# Calcular el porcentaje de error
merged_df['MAPE'] = (merged_df['abs_error'] / merged_df['amount'])

In [85]:
merged_df[merged_df['amount'] == 0]

,processing_date,pred_date,pred,payer_country,model,id_main_branch,id_country,payer,country,date,tx,amount,coupon_count,gp,day,abs_error,MAPE
45,2024-03-20,2023-12-26,0.000000,BANCO POPULAR HONDURAS (UT)_HONDURAS,MODEL_2d_BANCO POPULAR HONDURAS (UT)_HONDURAS.pkl,T304,HON,BANCO POPULAR HONDURAS (UT),HONDURAS,2023-12-26,0,0.0,0,0E-7,2023-12-26,0.000000,NaN
68,2024-03-20,2023-12-26,0.000000,BNB_SIERRA LEONE,MODEL_2d_BNB_SIERRA LEONE.pkl,T335,SLE,BNB,SIERRA LEONE,2023-12-26,0,0.0,0,-5.0000000,2023-12-26,0.000000,NaN
104,2024-03-20,2023-12-26,118.335448,FARMACIAS DEL AHORRO (DATRANSFER)_MEXICO,MODEL_2d_FARMACIAS DEL AHORRO (DATRANSFER)_MEX...,T257,MEX,FARMACIAS DEL AHORRO (DATRANSFER),MEXICO,2023-12-26,0,0.0,0,-3.2400000,2023-12-26,118.335448,inf
132,2024-03-20,2023-12-25,19503.400868,MERCHANTRADE_INDONESIA,MODEL_2d_MERCHANTRADE_INDONESIA.pkl,T201,IDN,MERCHANTRADE,INDONESIA,2023-12-25,0,0.0,0,2.0000000,2023-12-25,19503.400868,inf
143,2024-03-20,2023-12-26,361.096918,NAFA_BURKINA FASO,MODEL_2d_NAFA_BURKINA FASO.pkl,T303,BFA,NAFA,BURKINA FASO,2023-12-26,0,0.0,0,6.0000000,2023-12-26,361.096918,inf
184,2024-03-20,2023-12-26,0.000000,TRANSPAY_BANGLADESH,MODEL_2d_TRANSPAY_BANGLADESH.pkl,T298,BGD,TRANSPAY,BANGLADESH,2023-12-26,0,0.0,0,-6.0000000,2023-12-26,0.000000,NaN


In [89]:
mape_df = merged_df[['processing_date', 'pred_date', 'pred', 'payer_country',
            'id_main_branch', 'id_country', 'amount', 'abs_error', 'MAPE']]

In [90]:
mape_df

,processing_date,pred_date,pred,payer_country,id_main_branch,id_country,amount,abs_error,MAPE
0,2024-03-20,2023-12-25,36947.319758,24XORO_MEXICO,T314,MEX,2283.00,34664.319758,15.183671
1,2024-03-20,2023-12-26,43751.589258,24XORO_MEXICO,T314,MEX,42320.71,1430.879258,0.033810
2,2024-03-20,2023-12-25,9590.269894,ABANK (TN)_EL SALVADOR,T282,ELS,600.00,8990.269894,14.983783
3,2024-03-20,2023-12-26,8998.724809,ABANK (TN)_EL SALVADOR,T282,ELS,16501.00,7502.275191,0.454656
4,2024-03-20,2023-12-25,5632.096572,AFEX_CHILE,T089,CHI,280.00,5352.096572,19.114631
...,...,...,...,...,...,...,...,...,...
191,2024-03-20,2023-12-26,5351.794793,WALDOS (APPRIZA)_MEXICO,T244,MEX,5300.02,51.774793,0.009769
192,2024-03-20,2023-12-25,73846.434934,WALMART (UT)_MEXICO,T231,MEX,54096.24,19750.194934,0.365094
193,2024-03-20,2023-12-26,252735.429911,WALMART (UT)_MEXICO,T231,MEX,354927.14,102191.710089,0.287923
194,2024-03-20,2023-12-25,26430.191603,YES BANK_INDIA,T253,IND,4701.00,21729.191603,4.622249
